In [1]:
import pandas as pd
import numpy as np
import os
import kagglehub
from collections import namedtuple
import swifter

from pyproj import Transformer
from shapely import union_all
from shapely.ops import transform
import geopandas as gpd
import quackosm as qosm

import polars as pl

**Download the wine review dataset**

In [2]:
fname = "winemag-data-130k-v2.csv"
path = kagglehub.dataset_download("christopheiv/winemagdata130k")

**Load the wine review dataset into a polars dataframe**

In [3]:
reviews = pl.read_csv(os.path.join(path, fname))
reviews.schema


Schema([('', Int64),
        ('country', String),
        ('description', String),
        ('designation', String),
        ('points', Int64),
        ('price', Float64),
        ('province', String),
        ('region_1', String),
        ('region_2', String),
        ('taster_name', String),
        ('taster_twitter_handle', String),
        ('title', String),
        ('variety', String),
        ('winery', String)])

In [4]:
reviews.describe()

statistic,,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
str,f64,str,str,str,f64,f64,str,str,str,str,str,str,str,str
"""count""",129971.0,"""129908""","""129971""","""92506""",129971.0,120975.0,"""129908""","""108724""","""50511""","""103727""","""98758""","""129971""","""129970""","""129971"""
"""null_count""",0.0,"""63""","""0""","""37465""",0.0,8996.0,"""63""","""21247""","""79460""","""26244""","""31213""","""0""","""1""","""0"""
"""mean""",64985.0,null,null,null,88.447138,35.363389,null,null,null,null,null,null,null,null
"""std""",37519.540256,null,null,null,3.03973,41.022218,null,null,null,null,null,null,null,null
"""min""",0.0,"""Argentina""","""""Chremisa,"" the ancient name o…","""#19 Phantom Limb Belmont Viney…",80.0,4.0,"""Achaia""","""Abruzzo""","""California Other""","""Alexander Peartree""","""@AnneInVino""","""1+1=3 2008 Rosé Cabernet Sauvi…","""Abouriou""","""1+1=3"""
"""25%""",32493.0,null,null,null,86.0,17.0,null,null,null,null,null,null,null,null
"""50%""",64985.0,null,null,null,88.0,25.0,null,null,null,null,null,null,null,null
"""75%""",97478.0,null,null,null,91.0,42.0,null,null,null,null,null,null,null,null
"""max""",129970.0,"""Uruguay""","""“Wow” is the first word that c…","""“P”""",100.0,3300.0,"""Župa""","""Zonda Valley""","""Willamette Valley""","""Virginie Boone""","""@worldwineguys""","""Štoka 2011 Izbrani Teran (Kras…","""Žilavka""","""Štoka"""


In [5]:
location_cols=['winery', 'country', 'province', 'region_1', 'region_2']
reviews[location_cols]

winery,country,province,region_1,region_2
str,str,str,str,str
"""Nicosia""","""Italy""","""Sicily & Sardinia""","""Etna""",null
"""Quinta dos Avidagos""","""Portugal""","""Douro""",null,null
"""Rainstorm""","""US""","""Oregon""","""Willamette Valley""","""Willamette Valley"""
"""St. Julian""","""US""","""Michigan""","""Lake Michigan Shore""",null
"""Sweet Cheeks""","""US""","""Oregon""","""Willamette Valley""","""Willamette Valley"""
…,…,…,…,…
"""Dr. H. Thanisch (Erben Müller-…","""Germany""","""Mosel""",null,null
"""Citation""","""US""","""Oregon""","""Oregon""","""Oregon Other"""
"""Domaine Gresser""","""France""","""Alsace""","""Alsace""",null
